In [97]:
import numpy as np
import scipy
from scipy.io import loadmat
import mne, glob
from mne_features.feature_extraction import extract_features
import pandas as pd
from ipynb.fs.full.fullDataExtraction import getRawArrayData
from ipynb.fs.full.fullDataExtraction import extarctChannelNames
from ipynb.fs.full.feature_selection import get_features
import plotly.express as px
import plotly.offline as py
import plotly.graph_objects as go
import plotly
import math
from tensorflow import keras

Imported 0.2.03 version. Select nrows to a small number when running on huge datasets.
output = featurewiz(dataname, target, corr_limit=0.90, verbose=2, sep=',', 
		header=0, test_data='',feature_engg='', category_encoders='',
		dask_xgboost_flag=False, nrows=None, skip_sulov=False)
Create new features via 'feature_engg' flag : ['interactions','groupby','target']



In [100]:
def makeEpochs(rawArrays):
    sliced_epochs={}
    for clip in rawArrays:
        sliced_epochs[clip]= mne.make_fixed_length_epochs(rawArrays[clip], duration=1, preload=True,verbose=0)
    return sliced_epochs

def getRawArrays(matfile):
    del matfile["__header__"]
    del matfile["__version__"]
    del matfile["__globals__"]
    clip_info={}
    indexs=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
    keyName=list(matfile.keys())[0][:-1]
    channelNamesExtraction=extarctChannelNames(pd.read_excel("Preprocessed_EEG/channel-order.xlsx"))
    info=mne.create_info(channelNamesExtraction,200,'eeg')
    for ind,i in enumerate(indexs):
        rawData=matfile[keyName + str(i)]
        clip_info[ind] = mne.io.RawArray(rawData,info,verbose=0)
    return clip_info

def makeValueMatrix(currentAnalysed):
    result=pd.read_excel("Preprocessed_EEG/channel-order(topo).xlsx").to_numpy().astype(np.float32)
    #tracker value
    k=0
    for i in range(0,result.shape[0]):
        for j in range (0,result.shape[1]):
            if not(math.isnan(result[i][j])):
                result[i][j]=currentAnalysed[k]
                k+=1
    
    return result

def makePostionList(data):
    position=[]
    for column in range (0,500):
        position.append(makeValueMatrix(data[:,column]))
    return position

def ExtractFeatures(sliced_epochs):
    combined = np.zeros((1,len(features)*62))
    for cut in sliced_epochs:
        epoch_array=mne.Epochs.get_data(sliced_epochs[cut])
        extracted_data=extract_features(epoch_array,200,features)
        combined = np.vstack((combined,extracted_data))
    combined = np.delete(combined, 0, axis=0)
    return combined

In [101]:
channels=pd.read_excel("Preprocessed_EEG/channel-order(viz).xlsx")
channelNames=channels.iloc[:,0]
channelNames=np.ndarray.tolist(pd.Series.to_numpy(channelNames))
channelNames.insert(0,"Fp1")
ch_types = ['eeg'] * len(channelNames)

In [102]:
info = mne.create_info(channelNames, ch_types=ch_types, sfreq=200)
info.set_montage('standard_1020')

<Info | 8 non-empty values
 bads: []
 ch_names: Fp1, Fpz, Fp2, AF3, AF4, F7, F5, F3, F1, Fz, F2, F4, F6, F8, ...
 chs: 60 EEG
 custom_ref_applied: False
 dig: 63 items (3 Cardinal, 60 EEG)
 highpass: 0.0 Hz
 lowpass: 100.0 Hz
 meas_date: unspecified
 nchan: 60
 projs: []
 sfreq: 200.0 Hz
>

In [103]:
model = keras.models.load_model("emotionPredictionModel")
subject=loadmat('1_20131107.mat')
features = get_features()
rawArrays=getRawArrays(subject)
slicedEpochs=makeEpochs(rawArrays)
extractedData = ExtractFeatures(slicedEpochs)
points=makePostionList(data)

############################################################################################
############       F A S T   F E A T U R E  E N G G    A N D    S E L E C T I O N ! ########
# Be judicious with featurewiz. Don't use it to create too many un-interpretable features! #
############################################################################################
Correlation Limit = 0.7
Skipping feature engineering since no feature_engg input...
Skipping category encoding since no category encoders specified in input...
#### Single_Label Multi_Classification problem ####
    Loaded train data. Shape = (210429, 13)
#### Single_Label Multi_Classification problem ####
No test data filename given...
Classifying features using a random sample of 10000 rows from dataset...
#### Single_Label Multi_Classification problem ####
    loading a random sample of 10000 rows into pandas for EDA
#######################################################################################
##############

In [89]:
data=np.delete(subject['djc_eeg1'], 61, 0)
data=np.delete(data, 57, 0)
evoked=mne.EvokedArray(data,info)
transposedData=np.transpose(data)
df = pd.DataFrame(transposedData, columns = channelNames)
df = df.head(500)

In [118]:
y_pred = model.predict(extractedData)
y_pred = (y_pred > 0.5).astype(int)
y_pred = y_pred.tolist()

107/107 [==============================] - 0s 685us/step


In [124]:
from dash import Dash, dcc, html, Input, Output
from jupyter_dash import JupyterDash
app = JupyterDash(__name__)
slider = html.Div(
    [
        html.Label("Select Time Frame", htmlFor="time"),
        dcc.Slider(
            min=0,
            max=500,
            step=1,
            value=0,
            id="time",
            marks=None,
            tooltip={"placement": "bottom", "always_visible": True},
        ),
    ]
)
app.layout = html.Div(className='row', children=[
                 slider,
                 #container for linechart and heatMap
                 html.Div(
                    children=[
                    #heatmap div
                    html.Div(
                        children=[
                            dcc.Graph(id="heatmap"),
                    ],style={'display': 'inline-block'}),
                    #line chart div
                    html.Div(
                        children=[
                            dcc.Graph(id="lineChart")
                    ],style={'display': 'inline-block'}),
                    html.Div(id="emotion")
                    ])
                ])
@app.callback(Output("heatmap", "figure"),
              Output("lineChart", "figure"),
              Output("emotion", 'children'),
              Input("time", "value"))
def updateFigures(time_instance):
    heatMap = px.imshow(points[time_instance])
    heatMap.update_xaxes(showticklabels=False)
    heatMap.update_yaxes(showticklabels=False)
    heatMap.update_layout(xaxis=dict(scaleanchor='y', constrain='domain'),
                  width=550,
                  height=500 
                 )
    lineChart= px.line(df.head(time_instance))
    if (y_pred[time_instance] == [1,0,0]):
        emotion="positive"
    elif (y_pred[time_instance] == [0,1,0]):
        emotion="nuetral"
    else:
        emotion="negative"  
    emotionDiv=html.H1(emotion)
    return heatMap, lineChart, emotionDiv
if __name__ == "__main__":
    app.run_server()

Dash app running on http://127.0.0.1:8050/


In [119]:
y_pred[0]==[0,0,1]

True